In [2]:
import pandas as pd
import numpy as np

In [3]:
train_data = pd.read_csv('./features.csv', index_col='match_id')
test_data = pd.read_csv('./features_test.csv', index_col='match_id')

In [4]:
train_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


Признаки, которые мы будем использовать возьмем из тестовой выборки.

In [11]:
feature_columns_to_use = test_data.columns.tolist()

Проверим выюорку на наличие пропусков. Сразу выберем те, которые не равны числу строк во всей матрице - 97230:

In [14]:
train_data[feature_columns_to_use].count()[train_data[feature_columns_to_use].count() != 97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Видно, что некоторые признаки отсутствуют, причем в порядочном количестве. Но отсутсвие этих признаков легко объяснимо, так как мы рассматриваем только первые 5 минут игры. Например *first_blood_time*, *first_blood_team*, *first_blood_player1* и *first_blood_player2* могут отсутствовать, потому что за первые 5 минут не было первой крови. В признаке *first_blood_player2* еще больше пропущенных значений, видимо потому что если один из игроков все же завалил кого-то, для второго игрока это событие могло в 5 минут и не наступить. *radiant_bottle_time*, *radiant_courier_time*, *radiant_flying_courier_time* отсутствуют, потому что игроки не успели купить соответствующие предметы, так же *radiant_first_ward_time* может отсутствовать, если никто не успел установить "наблюдателя". Для Dire команды аналогично.

Соберем тестовые и тренировочные данные, для того чтобы проводить сразу все преобразования на тестовых и на тренировочных данных:

In [15]:
data = train_data[feature_columns_to_use].append(test_data[feature_columns_to_use])

Заменил пропущенные значения в данных на нули:

In [18]:
data = data.fillna(0)

Столбец, содержащий целевую переменную, это **radiant_win**. В нем содержится 1, если победила команда Radiant и 0 в противном случае.